In [1]:
import pandas as pd
import numpy as np
import nltk
import ast

In [2]:
import pickle
import re

In [19]:
hotels = pd.read_csv('zomato_outlet_final.csv')

In [20]:
hotels = hotels[['rest_name','rest_type','loc','dine_rating','dine_reviews','cuisine','cost','liked','link']]

In [81]:
hotels 

,rest_name,rest_type,loc,dine_rating,dine_reviews,cuisine,cost,liked,link,tags
0,Mix@36 - The Westin ...,[Lounge],"[TheWestin, Mundhwa]",[4.2],[707],"[NorthIndian, Continental, Asian]",[4300],"[Cocktails, Chaat, Beer]",https://www.zomato.com/pune/mix@36-the-westin-...,"[Lounge, TheWestin, Mundhwa, NorthIndian, Cont..."
1,Kangan - The Westin ...,[FineDining],"[TheWestin, Mundhwa]",[4.1],[344],"[NorthIndian, Mughlai, ]",[3500],"[Biryani, VegPlatter, DalMakhani]",https://www.zomato.com/pune/kangan-the-westin-...,"[FineDining, TheWestin, Mundhwa, NorthIndian, ..."
2,Coriander Kitchen - Conrad Pune ...,[FineDining],"[ConradPune, BundGardenRoad]",[4.5],[541],"[Mediterranean, Asian, NorthIndian]",[3200],"[Pasta, Panipuri, Waffles]",https://www.zomato.com/pune/coriander-kitchen-...,"[FineDining, ConradPune, BundGardenRoad, Medit..."
3,The Market - The Westin ...,[FineDining],"[TheWestin, Mundhwa]",[4.0],[55],"[Asian, European, NorthIndian]",[3200],"[VanillaIceCream, Chaat, Sushi]",https://www.zomato.com/pune/the-market-the-wes...,"[FineDining, TheWestin, Mundhwa, Asian, Europe..."
4,Vandaag - Vivanta Pune ...,"[Pub, CasualDining]","[VivantaPune, Hinjewadi]",[3.9],[113],"[FingerFood, Continental, ]",[3100],"[Cocktails, Beer, PeriPeriChicken]",https://www.zomato.com/pune/vandaag-vivanta-pu...,"[Pub, CasualDining, VivantaPune, Hinjewadi, Fi..."
...,...,...,...,...,...,...,...,...,...,...
1567,Awadh E Malwa ...,[QuickBites],[Bavdhan],[4.0],[190],"[Biryani, Awadhi, NorthIndian]",[600],"[ChickenMasala, CreamChicken, ButterChicken]",https://www.zomato.com/pune/awadh-e-malwa-bavdhan,"[QuickBites, Bavdhan, Biryani, Awadhi, NorthIn..."
1568,Anandi Dining Hall ...,[CasualDining],[SinhgadRoad],[3.6],[64],"[Maharashtrian, ]",[400],[VegThali],https://www.zomato.com/pune/anandi-dining-hall...,"[CasualDining, SinhgadRoad, Maharashtrian, , V..."
1569,The Irani Cafe Restaurant ...,[CasualDining],[VimanNagar],[2.9],[80],"[Mughlai, Iranian, ]",[800],"[Naan, MuttonKeema, DumBiryani]",https://www.zomato.com/pune/the-irani-cafe-res...,"[CasualDining, VimanNagar, Mughlai, Iranian, ,..."
1570,Hotel Shilpa Gavran ...,[CasualDining],[Pimpri],[3.8],[90],"[Chinese, FastFood, NorthIndian]",[500],"[MuttonThali, ChickenSukha]",https://www.zomato.com/pune/hotel-shilpa-gavra...,"[CasualDining, Pimpri, Chinese, FastFood, Nort..."


In [22]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5433 entries, 0 to 5432
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   rest_name     5433 non-null   object 
 1   rest_type     4336 non-null   object 
 2   loc           5433 non-null   object 
 3   dine_rating   4829 non-null   float64
 4   dine_reviews  4829 non-null   object 
 5   cuisine       5433 non-null   object 
 6   cost          5433 non-null   object 
 7   liked         2603 non-null   object 
 8   link          5433 non-null   object 
dtypes: float64(1), object(8)
memory usage: 382.1+ KB


In [23]:
hotels.isnull().sum()

rest_name          0
rest_type       1097
loc                0
dine_rating      604
dine_reviews     604
cuisine            0
cost               0
liked           2830
link               0
dtype: int64

In [24]:
hotels.drop_duplicates(inplace=True)

In [25]:

hotels.shape

(5211, 9)

In [26]:
duplicate_rows = hotels[hotels.duplicated(['rest_name','loc'], keep=False)]

In [27]:
print(duplicate_rows)

                                              rest_name      rest_type  \
777   PK Biryani House\r\n                          ...  Casual Dining   
799         KFC\r\n                                        Quick Bites   
800         KFC\r\n                                        Quick Bites   
807   McDonald's\r\n                                ...    Quick Bites   
809   McDonald's\r\n                                ...    Quick Bites   
...                                                 ...            ...   
5253  The Brooklyn Creamery\r\n                     ...            NaN   
5255  The Brooklyn Creamery\r\n                     ...            NaN   
5257  The Brooklyn Creamery\r\n                     ...            NaN   
5258  The Brooklyn Creamery\r\n                     ...            NaN   
5260  The Brooklyn Creamery\r\n                     ...            NaN   

                            loc  dine_rating dine_reviews  \
777                 Karve Nagar          4.1      

In [28]:
hotels = hotels.drop_duplicates(['rest_name','loc'])

--reset and shape--

In [29]:
duplicate_rows = hotels[hotels.duplicated(['rest_name'], keep=False)]

In [30]:
print(duplicate_rows)

                                              rest_name          rest_type  \
57    Little Italy\r\r\n                            ...      Casual Dining   
61    Little Italy\r\r\n                            ...      Casual Dining   
62    Little Italy\r\r\n                            ...      Casual Dining   
63    Little Italy\r\r\n                            ...  Casual Dining,Bar   
64    Little Italy\r\r\n                            ...                NaN   
...                                                 ...                ...   
5335  Khana Khajana\r\n                             ...        Quick Bites   
5341  Punjabi Rasoi\r\n                             ...        Quick Bites   
5352  Punjabi Rasoi\r\n                             ...      Casual Dining   
5353  Hotel Reddy\r\n                               ...        Quick Bites   
5417  Punjabi Rasoi\r\n                             ...                NaN   

                   loc  dine_rating   dine_reviews  \
57       

In [31]:
hotels = hotels.drop_duplicates(['rest_name'])

--reset and shape--

In [32]:
hotels.dropna(inplace=True)

In [33]:
hotels.reset_index(drop=True, inplace=True)

In [34]:
hotels.shape

(1572, 9)

In [35]:
hotels[['dine_rating','dine_reviews','cost']] = hotels[['dine_rating','dine_reviews','cost']].astype(str)

In [36]:
hotels['rest_type'][4]

'Pub,Casual Dining'

In [37]:
def reviewshandle(value):
    if 'Reviews' in value:
        value = value.replace("Reviews", "")
    if '(' in value:
        value = value.replace('(','')
    if ')' in value:
        value = value.replace(')','')
        return value
    else:
        return value    

In [38]:
hotels['dine_reviews'] = hotels['dine_reviews'].apply(reviewshandle)

In [39]:
hotels['dine_reviews'].unique

<bound method Series.unique of 0       707 
1       344 
2       541 
3        55 
4       113 
        ... 
1567     190
1568      64
1569     80 
1570     90 
1571      97
Name: dine_reviews, Length: 1572, dtype: object>

In [40]:
def costhandle(value):
    if 'Rs.' in value:
        value = value.replace('Rs.','')
    if ',' in value:
        value = value.replace(',','')
        return value
    else:
        return value

In [41]:
hotels['cost'] = hotels['cost'].apply(costhandle)

In [42]:
def convert3(obj):
    obj_list = obj.split(',')
    first_three = obj_list[:3]
    return first_three

In [43]:
hotels['rest_type'] 

0                  Lounge
1             Fine Dining
2             Fine Dining
3             Fine Dining
4       Pub,Casual Dining
              ...        
1567          Quick Bites
1568        Casual Dining
1569        Casual Dining
1570        Casual Dining
1571        Casual Dining
Name: rest_type, Length: 1572, dtype: object

In [44]:
hotels['liked'] = hotels['liked'].apply(convert3)

In [45]:
hotels['cuisine'] = hotels['cuisine'].apply(convert3)

In [46]:
#unused
def convert(obj):
    obj_list = obj.split(',')
    first=[] 
    first.append(obj_list[0])
    return first

In [47]:
def convertall(obj):
    obj_list = obj.split(',')

    return obj_list

In [48]:
hotels['loc'] = hotels['loc'].apply(convertall)

In [49]:
hotels['rest_type'] = hotels['rest_type'].apply(convertall)

In [50]:
def convert_num(obj):
    first=[] 
    first.append(obj)
    return first

In [51]:
hotels['dine_rating'] = hotels['dine_rating'].apply(convert_num)
hotels['dine_reviews'] = hotels['dine_reviews'].apply(convert_num)
hotels['cost'] = hotels['cost'].apply(convert_num)

In [52]:
#unused
import re

def add_space_before_newline(text):
    if '\r\r\n' in text:
        return re.sub(r'(\w)(\r\r\n)', r'\1 \2', text)
    elif '\r\n' in text:
        return re.sub(r'(\w)(\r\n)', r'\1 \2', text)
    else:
        return text

In [53]:
#unused
def namehandle(value):
    if '\r\r\n' in value:
        value = value.replace('\r\r\n', '')
    elif '\r\n' in value:
        value = value.replace('\r\n', '')
    else:
        return value

In [54]:
import re

def add_space_and_remove(text):
    modified_text = re.sub(r'(\w)(\r\r\n|\r\n)', r'\1 \2', text)
    return re.sub(r'\r\r\n|\r\n', '', modified_text)

In [55]:
hotels['rest_name'] = hotels['rest_name'].apply(add_space_and_remove)

In [56]:
hotels['cuisine'] = hotels['cuisine'].apply(lambda x:[i.replace(" ", "") for i in x])
hotels['liked'] = hotels['liked'].apply(lambda x:[i.replace(" ", "") for i in x])
hotels['loc'] = hotels['loc'].apply(lambda x:[i.replace(" ", "") for i in x])
# hotels['rest_name'] = hotels['rest_name'].apply(lambda x:x.replace(" ", ""))

In [57]:
hotels['rest_type'] = hotels['rest_type'].apply(lambda x:[i.replace(" ", "") for i in x])

In [58]:
hotels['dine_reviews'] = hotels['dine_reviews'].apply(lambda x:[i.replace(" ", "") for i in x])
hotels['cost'] = hotels['cost'].apply(lambda x:[i.replace(" ", "") for i in x])

In [59]:
hotels['tags'] = hotels['rest_type']+hotels['loc']+hotels['cuisine']+hotels['liked']+hotels['dine_rating']+hotels['dine_reviews']+hotels['cost']

In [60]:
hotels['tags'][0]

['Lounge',
 'TheWestin',
 'Mundhwa',
 'NorthIndian',
 'Continental',
 'Asian',
 'Cocktails',
 'Chaat',
 'Beer',
 '4.2',
 '707',
 '4300']

In [61]:
df = hotels[['rest_name','tags','link']]

In [62]:
df

,rest_name,tags,link
0,Mix@36 - The Westin ...,"[Lounge, TheWestin, Mundhwa, NorthIndian, Cont...",https://www.zomato.com/pune/mix@36-the-westin-...
1,Kangan - The Westin ...,"[FineDining, TheWestin, Mundhwa, NorthIndian, ...",https://www.zomato.com/pune/kangan-the-westin-...
2,Coriander Kitchen - Conrad Pune ...,"[FineDining, ConradPune, BundGardenRoad, Medit...",https://www.zomato.com/pune/coriander-kitchen-...
3,The Market - The Westin ...,"[FineDining, TheWestin, Mundhwa, Asian, Europe...",https://www.zomato.com/pune/the-market-the-wes...
4,Vandaag - Vivanta Pune ...,"[Pub, CasualDining, VivantaPune, Hinjewadi, Fi...",https://www.zomato.com/pune/vandaag-vivanta-pu...
...,...,...,...
1567,Awadh E Malwa ...,"[QuickBites, Bavdhan, Biryani, Awadhi, NorthIn...",https://www.zomato.com/pune/awadh-e-malwa-bavdhan
1568,Anandi Dining Hall ...,"[CasualDining, SinhgadRoad, Maharashtrian, , V...",https://www.zomato.com/pune/anandi-dining-hall...
1569,The Irani Cafe Restaurant ...,"[CasualDining, VimanNagar, Mughlai, Iranian, ,...",https://www.zomato.com/pune/the-irani-cafe-res...
1570,Hotel Shilpa Gavran ...,"[CasualDining, Pimpri, Chinese, FastFood, Nort...",https://www.zomato.com/pune/hotel-shilpa-gavra...


In [63]:
df['tags'] = df['tags'].apply(lambda x:" ".join(x)) 

/var/folders/lp/3pg1pm7x4fq16yzd6ncss4lc0000gr/T/ipykernel_21495/863369503.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x:" ".join(x))


In [64]:
df['tags'] = df['tags'].apply(lambda x:x.lower()) 

/var/folders/lp/3pg1pm7x4fq16yzd6ncss4lc0000gr/T/ipykernel_21495/1119052810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x:x.lower())


In [65]:
df['tags'][0]

'lounge thewestin mundhwa northindian continental asian cocktails chaat beer 4.2 707 4300'

In [66]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2182, stop_words='english')

In [67]:
vectors = cv.fit_transform(df['tags']).toarray()

In [68]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [69]:
len(vectors)

1572

In [70]:
featurenames=cv.get_feature_names_out()
print(featurenames)

['001' '004' '005' ... 'yerawada' 'zaatar' 'zucchinisoup']


In [71]:
from sklearn.metrics.pairwise import cosine_similarity

In [72]:
similarity = cosine_similarity(vectors)

In [73]:
#re lib

def normalize_name(name):
    # Remove extra spaces and convert to lowercase
    return re.sub(r'[^\w\s]', '', name).strip().lower()

In [74]:
def recommend(hotel):
    hotel = normalize_name(hotel)
    hotel_index = df[df['rest_name'].apply(normalize_name).str.contains(hotel)].index[0]
    distances = similarity[hotel_index]
    hotels_list = sorted(list(enumerate(distances)),reverse=True,key = lambda x:x[1])[1:6]
    
    for i in hotels_list:
        print(df.iloc[i[0]].rest_name)

In [80]:
recommend('vaishali')

Wadeshwar                                     
Shree Krishna Veg Court                                     
Wagh Bakri Tea Lounge                                     
Niranjan Veg Restaurant                                     
Supriya Restaurant                                     


In [76]:
pickle.dump(df.to_dict(), open('hotel_dict.pkl','wb'))

In [77]:
pickle.dump(similarity, open('hotel_similarity.pkl','wb'))

Evaluation